In [82]:
!pip install pyspark

In [83]:
!pip install pymongo 

#### **Setting up a Spark session**

In [84]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [85]:
spark = SparkSession.builder.master("local").appName("Gender_Gap_Analysis").getOrCreate()

In [86]:
rdd = spark.read.csv("/content/PanelStudyIncomeDynamics.csv",inferSchema=True,header=True)

In [87]:
rdd.show(10)

+--------+--------+----+---+------+-------+------+----------------+-------+---+--------+---+------+---------+---+---+-----+-----+----+-------+------+------+------+----------------+-----+---------+---------+------+--------+--------+----------+----------+--------+--------+----------+----------+----------+------------+------------+------+-------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+-------+------------+------------+---------------+---------------+----------------+----------------+--------------+--------------+------------+------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+-----------+---------+---------+---------+-----------+---------+---------+----------+----------+----------+-----------+-------+--------------+--------------+--------------+--------------+-----

In [88]:
print("Number of rows in the dataframe:",rdd.count())
print('Number of columns in the dataframe:',len(rdd.columns))

Number of rows in the dataframe: 33398
Number of columns in the dataframe: 274


In [89]:
from pyspark.sql.functions import col,isnan,when,count
rdd.select([count(when(col(c).contains('None') | col(c).contains('NULL') | \
                            (col(c) == '' ) |col(c).isNull() , c )).alias(c)
                    for c in rdd.columns]).show()

+--------+--------+----+---+------+-------+------+------+-------+---+--------+---+------+---------+---+---+-----+-----+----+-------+------+------+------+------+-----+---------+---------+------+--------+--------+----------+----------+--------+--------+----------+----------+----------+------------+------------+------+-------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+---------+-----------+-------+------------+------------+---------------+---------------+--------------+--------------+--------------+--------------+------------+------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+-----------+---------+---------+---------+-----------+---------+---------+----------+----------+----------+-----------+-------+--------------+--------------+--------------+--------------+--------------+--------------+-----

In [90]:
rdd = rdd.drop('predict98' , 'predictft98' , 'predict00' , 'predictft00' , 'predict02' , 'predictft02' ,
             'predict04' , 'predictft04' , 'predict06' , 'predictft06' , 'predict08' , 'predictft08' , 
             'predict10' , 'predictft10' , 'predict97' , 'predictft97' , 'predictfz97' , 'predictftfz97',
             'predictfz98' , 'predictftfz98' , 'predict1999' , 'predictft1999' , 'predictfz99',
             'predictftfz99' , 'predictfz00' , 'predictftfz00' , 'predict01' , 'predictft01' , 'predictfz01',
             'predictftfz01' , 'predictfz02' , 'predictftfz02' , 'predict03' , 'predictft03' , 'predictfz03' ,
             'predictftfz03' , 'predictfz04' , 'predictftfz04' , 'predictfz06' , 'predictftfz06',
             'predict2007' , 'predictft2007' , 'predictfz07' , 'predictftfz07' , 'predictfz08' , 'predictftfz08',
             'predict2009' , 'predictft2009' , 'predictfz09' , 'predictftfz09' , 'predictfz10' , 'predictftfz10',
             'predict2011' , 'predictft2011' , 'predictfz11' , 'predictftfz11')

In [91]:
print("Number of rows in the dataframe:",rdd.count())
print('Number of columns in the dataframe:',len(rdd.columns))

Number of rows in the dataframe: 33398
Number of columns in the dataframe: 218


In [92]:
rdd=rdd.na.drop(how = "any")

In [93]:
rdd.dtypes

[('intnum68', 'int'),
 ('pernum68', 'int'),
 ('wave', 'int'),
 ('sex', 'int'),
 ('intnum', 'int'),
 ('farminc', 'double'),
 ('region', 'string'),
 ('famwgt', 'double'),
 ('relhead', 'int'),
 ('age', 'int'),
 ('employed', 'int'),
 ('sch', 'string'),
 ('annhrs', 'int'),
 ('annlabinc', 'double'),
 ('occ', 'int'),
 ('ind', 'int'),
 ('white', 'int'),
 ('black', 'int'),
 ('hisp', 'int'),
 ('othrace', 'int'),
 ('degree', 'string'),
 ('degupd', 'string'),
 ('schupd', 'int'),
 ('annwks', 'double'),
 ('unjob', 'int'),
 ('usualhrwk', 'string'),
 ('labincbus', 'int'),
 ('yrsexp', 'double'),
 ('yrsftexp', 'double'),
 ('yrsptexp', 'double'),
 ('yrsptexpsq', 'double'),
 ('yrsftexpsq', 'double'),
 ('yrsExpSq', 'double'),
 ('yrsexpfz', 'double'),
 ('yrsftexpfz', 'double'),
 ('yrsptexpfz', 'double'),
 ('yrsexpfzsq', 'double'),
 ('yrsftexpfzsq', 'double'),
 ('yrsptexpfzsq', 'double'),
 ('wtrgov', 'string'),
 ('selfemp', 'int'),
 ('origage', 'int'),
 ('origannHrsHD', 'int'),
 ('origannHrsWF', 'int'),
 ('o

In [94]:
from pyspark.sql.functions import desc
rdd.groupBy('sex').count().sort(desc('count')).show()

+---+-----+
|sex|count|
+---+-----+
|  2|17162|
|  1|16236|
+---+-----+



In [95]:
dt = rdd.dtypes
columnList = [item[0] for item in dt if item[1].startswith(
    'string') | item[1].startswith('bigint')]


In [96]:
orig_col = [item for item in columnList if item.startswith('orig')]


In [97]:
rdd = rdd.drop('origannWeeksHD',
 'origannWeeksWF',
 'origdegreeHD',
 'origdegreeWF',
 'origeverwrkHD07',
 'origeverwrkHD09',
 'origeverwrkHD11',
 'origeverwrkHD99',
 'origeverwrkWF07',
 'origeverwrkWF09',
 'origeverwrkWF11',
 'origeverwrkWF99',
 'origindHD',
 'origindWF',
 'origoccHD',
 'origoccWF',
 'origraceHD',
 'origraceWF',
 'origschfamHD07',
 'origschfamHD09',
 'origschfamHD11',
 'origschfamHD81',
 'origschfamHD99',
 'origschfamWF07',
 'origschfamWF09',
 'origschfamWF11',
 'origschfamWF81',
 'origschfamWF99',
 'origspanHD',
 'origspanWF',
 'origstopwrkHDE299',
 'origstopwrkHDE399',
 'origstopwrkHDE499',
 'origstopwrkHDE599',
 'origstopwrkHDE699',
 'origstopwrkHDU299',
 'origstopwrkHDU399',
 'origstopwrkHDU499',
 'origstopwrkHDU599',
 'origstopwrkHDU699',
 'origstopwrkWFE299',
 'origstopwrkWFE399',
 'origstopwrkWFE499',
 'origstopwrkWFE599',
 'origstopwrkWFE699',
 'origstopwrkWFU299',
 'origstopwrkWFU399',
 'origstopwrkWFU499',
 'origstopwrkWFU599',
 'origstopwrkWFU699',
 'origwrkPriorJanHD',
 'origwrkPriorJanWF',
 'origyrNewHD',
 'origyrNewWF',
 'origAdvHD',
 'origAdvWF',
 'origBAHD',
 'origBAWF',
 'origannWeeksHDE',
 'origannWeeksHDR',
 'origannWeeksHDU',
 'origannWeeksWFE',
 'origannWeeksWFR',
 'origannWeeksWFU',
 'origindHDE',
 'origindWFE',
 'origindHDU',
 'origindWFU',
 'origindHDR',
 'origindWFR',
 'origoccHDE',
 'origoccHDR',
 'origoccHDU',
 'origoccWFE',
 'origoccWFR',
 'origoccWFU',
 'origrace',
 'origschHD',
 'origschWF',
 'origyrHghstDegHD',
 'origyrHghstDegWF',
 'origwtrCollDegHD',
 'origwtrCollDegWF',
 'origwtrCollHD',
 'origwtrCollWF')

In [98]:
print("Number of rows in the dataframe:",rdd.count())
print('Number of columns in the dataframe:',len(rdd.columns))

Number of rows in the dataframe: 33398
Number of columns in the dataframe: 133


In [99]:
rdd.dtypes

[('intnum68', 'int'),
 ('pernum68', 'int'),
 ('wave', 'int'),
 ('sex', 'int'),
 ('intnum', 'int'),
 ('farminc', 'double'),
 ('region', 'string'),
 ('famwgt', 'double'),
 ('relhead', 'int'),
 ('age', 'int'),
 ('employed', 'int'),
 ('sch', 'string'),
 ('annhrs', 'int'),
 ('annlabinc', 'double'),
 ('occ', 'int'),
 ('ind', 'int'),
 ('white', 'int'),
 ('black', 'int'),
 ('hisp', 'int'),
 ('othrace', 'int'),
 ('degree', 'string'),
 ('degupd', 'string'),
 ('schupd', 'int'),
 ('annwks', 'double'),
 ('unjob', 'int'),
 ('usualhrwk', 'string'),
 ('labincbus', 'int'),
 ('yrsexp', 'double'),
 ('yrsftexp', 'double'),
 ('yrsptexp', 'double'),
 ('yrsptexpsq', 'double'),
 ('yrsftexpsq', 'double'),
 ('yrsExpSq', 'double'),
 ('yrsexpfz', 'double'),
 ('yrsftexpfz', 'double'),
 ('yrsptexpfz', 'double'),
 ('yrsexpfzsq', 'double'),
 ('yrsftexpfzsq', 'double'),
 ('yrsptexpfzsq', 'double'),
 ('wtrgov', 'string'),
 ('selfemp', 'int'),
 ('origage', 'int'),
 ('origannHrsHD', 'int'),
 ('origannHrsWF', 'int'),
 ('o

In [100]:
# Check colerration for numerical columns 
import six
for i in rdd.columns:
    if not( isinstance(rdd.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to annualinc for ", i, rdd.stat.corr('annlabinc',i))

Correlation to annualinc for  intnum68 -0.17328146362222996
Correlation to annualinc for  pernum68 0.06111082227263366
Correlation to annualinc for  wave 0.2971478148302535
Correlation to annualinc for  sex -0.19991963725187012
Correlation to annualinc for  intnum 0.052635586353324944
Correlation to annualinc for  farminc -0.011258856484110792
Correlation to annualinc for  famwgt 0.13943216586728988
Correlation to annualinc for  relhead -0.12888845327597356
Correlation to annualinc for  age 0.14694283572364253
Correlation to annualinc for  employed nan
Correlation to annualinc for  annhrs 0.2953599318635706
Correlation to annualinc for  annlabinc 1.0
Correlation to annualinc for  occ -0.22515617284419429
Correlation to annualinc for  ind -0.025029767584991938
Correlation to annualinc for  white 0.14729346618605313
Correlation to annualinc for  black -0.15385904647695609
Correlation to annualinc for  hisp -0.0011034720741645715
Correlation to annualinc for  othrace 0.009440989337241038


In [101]:
rdd.select('region').distinct().collect()

[Row(region='3'),
 Row(region='NA'),
 Row(region='1'),
 Row(region='4'),
 Row(region='2')]

In [102]:
cols = rdd.columns
cat_cols=[i[0] for i in rdd.dtypes if i[1].startswith('string')]
num_cols=[i[0] for i in rdd.dtypes if i[1].startswith(('int','double'))]

In [103]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler
stages = []
for i in cat_cols:
    stringIndexer = StringIndexer(inputCol = i, outputCol = i + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[i + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'annlabinc', outputCol = 'label')
stages += [label_stringIdx]
assemblerInputs = [c + "classVec" for c in cat_cols] + num_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [104]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(rdd)
df = pipelineModel.transform(rdd)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- intnum68: integer (nullable = true)
 |-- pernum68: integer (nullable = true)
 |-- wave: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- intnum: integer (nullable = true)
 |-- farminc: double (nullable = true)
 |-- region: string (nullable = true)
 |-- famwgt: double (nullable = true)
 |-- relhead: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- employed: integer (nullable = true)
 |-- sch: string (nullable = true)
 |-- annhrs: integer (nullable = true)
 |-- annlabinc: double (nullable = true)
 |-- occ: integer (nullable = true)
 |-- ind: integer (nullable = true)
 |-- white: integer (nullable = true)
 |-- black: integer (nullable = true)
 |-- hisp: integer (nullable = true)
 |-- othrace: integer (nullable = true)
 |-- degree: string (nullable = true)
 |-- degupd: string (nullable = true)
 |-- schupd: integer (nullable = true)
 |-- annwks: double (nullable = true

In [105]:
train_df, test_df = df.randomSplit([0.7, 0.3], seed = 42)
print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))

Training Dataset Count: 23453
Test Dataset Count: 9945


In [106]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="label")

In [107]:
lrmodel = lr.fit(train_df)

In [108]:
lr_m=lrmodel.transform(test_df)

In [109]:
lr_a=lrmodel.summary
lr_a

In [110]:
lr_a.rootMeanSquaredError

1011.3775231897075